# Imports

In [2]:
import plotly.express as ex
import pandas as pd
import numpy as np
import math, sys, os
from collections import Counter
sys.path.append('../')
from covid_moonshot_ml.plotting import plotting as pl
fn = "/Users/alexpayne/Scientific_Projects/local_test/datasets/sars-redocked.csv"
df = pd.read_csv(fn).fillna("-1")

In [3]:
df

,Complex_ID,Compound_ID,Crystal_ID,Chain_ID,MCSS_Rank,SDF_Filename,Reference,POSIT,Chemgauss4,RMSD
0,ADA-UCB-6c2cb422-1_Mpro-P2028_0B_0,ADA-UCB-6c2cb422-1,Mpro-P2028,0B,0,kinoml_OEDockingFeaturizer_MPRO_Mpro-P2028_0B_...,Mpro-P2005_0A/Mpro-P2005_0A_bound.pdb,0.72,-9.152148,0.288372
1,ADA-UCB-6c2cb422-1_Mpro-P2185_0B_9,ADA-UCB-6c2cb422-1,Mpro-P2185,0B,9,kinoml_OEDockingFeaturizer_MPRO_Mpro-P2185_0B_...,Mpro-P2005_0A/Mpro-P2005_0A_bound.pdb,0.85,-9.470764,0.374129
2,ADA-UCB-6c2cb422-1_Mpro-P0764_0A_1,ADA-UCB-6c2cb422-1,Mpro-P0764,0A,1,kinoml_OEDockingFeaturizer_MPRO_Mpro-P0764_0A_...,Mpro-P2005_0A/Mpro-P2005_0A_bound.pdb,0.72,17.041128,1.696102
3,ADA-UCB-6c2cb422-1_Mpro-P0098_0A_3,ADA-UCB-6c2cb422-1,Mpro-P0098,0A,3,kinoml_OEDockingFeaturizer_MPRO_Mpro-P0098_0A_...,Mpro-P2005_0A/Mpro-P2005_0A_bound.pdb,0.85,18.198183,0.830546
4,ADA-UCB-6c2cb422-1_Mpro-P2205_0A_7,ADA-UCB-6c2cb422-1,Mpro-P2205,0A,7,kinoml_OEDockingFeaturizer_MPRO_Mpro-P2205_0A_...,Mpro-P2005_0A/Mpro-P2005_0A_bound.pdb,0.23,-7.522447,1.435679
...,...,...,...,...,...,...,...,...,...,...
3987,VLA-UNK-cf7facf1-1_Mpro-P0022_0A_1,VLA-UNK-cf7facf1-1,Mpro-P0022,0A,1,kinoml_OEDockingFeaturizer_MPRO_Mpro-P0022_0A_...,Mpro-P0143_0B/Mpro-P0143_0B_bound.pdb,0.99,21.098890,46.820838
3988,VLA-UNK-cf7facf1-1_Mpro-P0811_0B_4,VLA-UNK-cf7facf1-1,Mpro-P0811,0B,4,kinoml_OEDockingFeaturizer_MPRO_Mpro-P0811_0B_...,Mpro-P0143_0B/Mpro-P0143_0B_bound.pdb,0.90,-9.516819,47.030816
3989,VLA-UNK-cf7facf1-1_Mpro-P0143_0A_-1,VLA-UNK-cf7facf1-1,Mpro-P0143,0A,-1,kinoml_OEDockingFeaturizer_MPRO_Mpro-P0143_0A_...,Mpro-P0143_0B/Mpro-P0143_0B_bound.pdb,0.99,21.594009,46.778248
3990,VLA-UNK-cf7facf1-1_Mpro-P0017_0A_9,VLA-UNK-cf7facf1-1,Mpro-P0017,0A,9,kinoml_OEDockingFeaturizer_MPRO_Mpro-P0017_0A_...,Mpro-P0143_0B/Mpro-P0143_0B_bound.pdb,0.72,20.980835,46.702409


In [10]:
df["POSIT_R"] = 1-df.POSIT

In [21]:
rocks = pl.Rocks(fn, ["MCSS_Rank", "POSIT_R", "Chemgauss4"], 
                 "RMSD", 
                 100,
                100)

In [22]:
rocks.clean_dataframe()

In [23]:
ex.histogram(rocks.df, x="RMSD")

In [24]:
rocks.get_aucs()

In [25]:
rocks.combine_dfs()

In [26]:
rocks.plot_poses_auc()

In [28]:
rocks.get_auc_cis()

MCSS_Rank
POSIT_R
Chemgauss4


In [29]:
rocks.model_df

,Score_Type,Lower_Bound,AUC,Upper_Bound
0,MCSS_Rank,0.000000e+00,0.619262,0.000000e+00
1,POSIT_R,-1.110223e-16,0.893318,1.110223e-16
2,Chemgauss4,1.110223e-16,0.546209,-1.110223e-16


In [32]:
ex.bar(rocks.model_df, 
       x="Score_Type", 
       y="AUC", 
       error_y="Upper_Bound", 
       error_y_minus="Lower_Bound")

In [31]:
rocks.plot_precision_recall()

In [33]:
ex.scatter(rocks.df, 
           x="POSIT_R", 
           y="RMSD", 
           color="Chemgauss4", 
           hover_data=["Compound_ID",
                       "Crystal_ID", 
                       "Reference", 
                       "Chain_ID"], 
          )

In [36]:
ex.density_heatmap(rocks.df,
                  x="POSIT_R",
                  y="RMSD")